<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 5 - Adding Regularization with L2 and Dropout - Aladdin Persson anlattı.
</font>

**Video:** [TensorFlow Tutorial 5 - Adding Regularization with L2 and Dropout](https://www.youtube.com/watch?v=kJSUq1PLmWg&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=5)

*Önce [TensorFlow Tutorial 4 - Convolutional Neural Networks with Sequential and Functional API](https://www.youtube.com/watch?v=WAciKiDP2bo&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=4) videosu izlenmeli.*

### İçindekiler

**Loading Dataset**

**Preprocessing Dataset**

**Creating Convolutional Neural Network**
* L2 regularization, dropout, batch norm eklendi.

### Kaynaklar

* Regularization, dropout, early stopping konularını teorik olarak hatırlamak için [Intro to Deep Learning with PyTorch (Udacity)](https://docs.google.com/document/d/1HqIl6KYr0F4NGgPiYYhWw-On_muAj4mxJm16nAQXL5M/edit?usp=sharing) notlarına bakabilirsin.

**Tekrar çalıştırma!**

### <font color="blue"> Giriş</font>

Önceki notebookta training accuracy ve test accuracymiz arasında ciddi bir fark olduğunu görmüştük. Bunun nedeni çalıştırdığımız modelin training datada overfitting yapmasından kaynaklıydı. Overfittingi **regularization** ile azaltıyoruz. Birçok regularization metodu var. 

* You can reduce your model capacity to avoid overfitting,
* You can also add L2 regularization, dropout, early stopping or adding data augmentation.

Bu notebookta nasıl **L2 regularization ve dropout** eklediğimize bakacağız.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers # ek olarak regularizersı ekledik.

### 1. Loading Dataset

In [2]:
from tensorflow.keras.datasets import cifar10

In [3]:
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


### 2. Preprocessing Dataset

1. `x_train.astype("float32")` koduyla **computationı minimize etmek** için type'ı float64'ten float32'ye dönüştürüyoruz.


2. `x_train.astype("float32") / 255.0` ile piksel değerlerini **normalize** ediyoruz. 


3. Aynı işlemi `x_test = x_test.astype("float32") / 255.0` koduyla **test seti için de** yapıyoruz. 

In [5]:
x_train = x_train.astype("float32") / 255.0 

In [6]:
x_test = x_test.astype("float32") / 255.0 

### 3. Creating Convolutional Neural Network

<font color="black"> 3.2. Functional API Kullanarak yazdığımız modelimizde L2 regularization ve dropout ekledik.</font>

##### <font color="green"> `my_model` fonksiyonu tanımlayıp layerları yazıyoruz. </font>

In [7]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3)) 
    x = layers.Conv2D(32,3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
    x = layers.BatchNormalization()(x)   
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 5, padding='same', kernel_regularizer=regularizers.l2(0.01))(x) 
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

model = my_model()

<font color="black">Her layer için ayrı ayrı regularization ekleyeceğiz. 
    
* İlki için L2 regularizerı eklemeye karar verdik. İlk layerımızı `x = layers.Conv2D(32,3)(inputs)` şeklinde tanımlamıştık. Burada basitlik olsun diye `padding = 'same'` olarak belirledik, default olarak 'valid' oluyordu normalde. `kernel_regularizer=regularizers.l2(0.01)` ile bu spesifik layer için regularization eklemiş olduk. Bunu diğer tüm layerlar için yapacağız. 


* İki fully connected layer arasına `x = layers.Dropout(0.5)` kodu ile dropout ekledik. Yani bir önceki layer ile 10 nodluk layer arasındaki connectionsların yarısını drop edecek.
    
    
* Batchnormalizationların amacı normalde datayı daha hızlı eğitmek için normalize etmek ama aynı zamanda regularization etkisi var. 

Burada 3 farklı metod kullanmış olduk regularization için. **L2, dropout ve batch norm**.

</font>

##### <font color="green"> `model.compile()` içerisinde ağımızın eğitim bölümünü nasıl yapılandıracağımızı anlatıyoruz kerasa. </font>

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics = ["accuracy"],
)

##### <font color="green"> `model.fit()` ile yapılandırdığımız modeli somut olarak eğitiyoruz. </font>

Dropout eklendiğinde training daha uzun sürer. Çünkü batchlerin arasındaki connectionsları drop ediyoruz. Bu yüzden 10 epochda eğitmek yerine 150 epochta eğiteceğiz. 

In [9]:
model.fit(x_train, y_train, batch_size=64, epochs=150, verbose=2)

Epoch 1/150
782/782 - 300s - loss: 3.1561 - accuracy: 0.1561
Epoch 2/150
782/782 - 272s - loss: 2.2363 - accuracy: 0.1915
Epoch 3/150
782/782 - 267s - loss: 2.0899 - accuracy: 0.2233
Epoch 4/150
782/782 - 255s - loss: 2.0239 - accuracy: 0.2385
Epoch 5/150
782/782 - 254s - loss: 1.9996 - accuracy: 0.2457
Epoch 6/150
782/782 - 254s - loss: 1.9723 - accuracy: 0.2520
Epoch 7/150
782/782 - 261s - loss: 1.9531 - accuracy: 0.2565
Epoch 8/150
782/782 - 254s - loss: 1.9437 - accuracy: 0.2582
Epoch 9/150
782/782 - 253s - loss: 1.9282 - accuracy: 0.2646
Epoch 10/150
782/782 - 254s - loss: 1.9260 - accuracy: 0.2639
Epoch 11/150
782/782 - 299s - loss: 1.9161 - accuracy: 0.2649
Epoch 12/150
782/782 - 288s - loss: 1.9136 - accuracy: 0.2664
Epoch 13/150
782/782 - 316s - loss: 1.9090 - accuracy: 0.2691
Epoch 14/150
782/782 - 303s - loss: 1.8989 - accuracy: 0.2681
Epoch 15/150
782/782 - 306s - loss: 1.8981 - accuracy: 0.2686
Epoch 16/150
782/782 - 319s - loss: 1.8893 - accuracy: 0.2703
Epoch 17/150
782/

Epoch 133/150
782/782 - 258s - loss: 1.7917 - accuracy: 0.3108
Epoch 134/150
782/782 - 244s - loss: 1.7857 - accuracy: 0.3127
Epoch 135/150
782/782 - 268s - loss: 1.7850 - accuracy: 0.3123
Epoch 136/150
782/782 - 313s - loss: 1.7916 - accuracy: 0.3127
Epoch 137/150
782/782 - 297s - loss: 1.7816 - accuracy: 0.3146
Epoch 138/150
782/782 - 340s - loss: 1.7869 - accuracy: 0.3130
Epoch 139/150
782/782 - 283s - loss: 1.7964 - accuracy: 0.3099
Epoch 140/150
782/782 - 293s - loss: 1.7839 - accuracy: 0.3130
Epoch 141/150
782/782 - 258s - loss: 1.7898 - accuracy: 0.3130
Epoch 142/150
782/782 - 273s - loss: 1.7837 - accuracy: 0.3140
Epoch 143/150
782/782 - 279s - loss: 1.7881 - accuracy: 0.3121
Epoch 144/150
782/782 - 307s - loss: 1.7866 - accuracy: 0.3116
Epoch 145/150
782/782 - 321s - loss: 1.7853 - accuracy: 0.3133
Epoch 146/150
782/782 - 293s - loss: 1.7768 - accuracy: 0.3148
Epoch 147/150
782/782 - 266s - loss: 1.7798 - accuracy: 0.3153
Epoch 148/150
782/782 - 271s - loss: 1.7848 - accuracy:

##### <font color="green"> `model.evaluate()` ile test setimizi gönderip modelimizin başarısını değerlendiriyoruz. </font>

In [10]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 9s - loss: 1.6884 - accuracy: 0.3752


[1.6884212493896484, 0.3752000033855438]

Bir önceki notebookta training accuracy 0.92 iken test accuracy 0.67 çıktı. Burada ise training accuracy 0.31 iken test accuracy 0.37 oldu. Aradaki fark azaldı. Training accuracymiz düşse de overfittingi engellediğimizi söyleyebiliriz. Epoch sayısını daha çok artırırsak (daha uzun süre eğitirsek) başarımız da artacaktır.